In [1]:
!pip install pyspark

In [8]:
from pyspark import SparkContext

# Crear el contexto de Spark
sc = SparkContext("local", "ShortestPath")

# Definir la función para enviar mensajes entre nodos
def enviar(vecinos):
    mensaje = []
    for vecino in vecinos:
        mensaje.append((vecino[1][0] , vecino[0] + vecino[1][1]))
    return mensaje

# [(2, [(inf, (3, 3)), (inf, (4, 24))]), (4, [(inf, (5, 5)), (inf, (6, 2))]), (1, [(0, (2, 10))]), (3, [(inf, (2, 1))]), (5, [(inf, (6, 7))])]

# Definir la función para realizar el merge de los mensajes recibidos por cada nodo
def unir(cost1, cost2):
    return min(cost1, cost2)

# Definir la función para actualizar el costo asociado a cada nodo
def actualizar(nodo, mensaje):
    return (nodo, mensaje)

# Definir la función para verificar si el costo de los nodos ha convergido
def convergencia(nuevo_costo, viejo_costo):
    return nuevo_costo.join(viejo_costo).mapValues(lambda x: x[0] == x[1]).values().reduce(lambda x, y: x and y)

# Definir la función para encontrar el camino más corto desde el nodo inicial
def sssp(nodos, aristas, nodo_inicial):
    # Crear RDD con los nodos y sus costos iniciales
    costo_inicial = float("inf")
    costos_iniciales = sc.parallelize([(nodo, costo_inicial) if nodo != nodo_inicial else (nodo, 0) for nodo in nodos])

    # Convertir la lista de aristas en un RDD
    aristas_rdd = sc.parallelize(aristas).map(lambda x: (x[0], (x[1], x[2])))

    # Inicializar los valores de los costos
    viejo_costo = costos_iniciales
    nuevo_costo = costos_iniciales

    iteracion = 0

    while iteracion < 2:
        # Enviar mensajes y hacer el merge
        mensajes = nuevo_costo.join(aristas_rdd).groupByKey().flatMapValues(enviar).map(lambda x: x[1]) # [(3, inf), (4, inf), (5, inf), (6, inf), (2, 10), (2, inf), (6, inf)]

        mensaje_recibido = mensajes.reduceByKey(unir) # [(4, inf), (6, inf), (2, 10), (3, inf), (5, inf)]

        # Actualizar los costos
        nuevo_costo = nuevo_costo.leftOuterJoin(mensaje_recibido).mapValues(lambda x: 0 if x[1] is None else x[1]) # [(3, (inf, 4)), (6, inf), (1, (0, 4)), (4, (inf, 10)), (2, (inf, 6)), (5, (inf, 12))]

        if convergencia(nuevo_costo, viejo_costo):
          iteracion += 1

        # Actualizar los valores de los costos antiguos
        viejo_costo = nuevo_costo

    return nuevo_costo.collect()

# Definir los nodos y aristas del grafo
nodos = [1, 2, 3, 4, 5, 6]
aristas = [(1, 2, 10), (2, 3, 3), (2, 4, 24), (3, 2, 1), (4, 5, 5), (4, 6, 2), (5, 6, 7)]

# Definir el nodo inicial
nodo_inicial = 1

# Encontrar el camino más corto desde el nodo inicial
resultado = sssp(nodos, aristas, nodo_inicial)

# Imprimir los resultadoados
for nodo, cost in resultado:
    print(f"Nodo {nodo}: Costo = {cost}")

# Detener el contexto de Spark
sc.stop()


0
[(3, inf), (6, inf), (1, 0), (4, inf), (2, 10), (5, inf)]
0
[(1, 0), (2, 10), (3, 13), (4, 34), (5, inf), (6, inf)]
0
[(1, 0), (2, 10), (3, 13), (4, 34), (5, 39), (6, 36)]
1
[(1, 0), (2, 10), (3, 13), (4, 34), (5, 39), (6, 36)]
2
[(1, 0), (2, 10), (3, 13), (4, 34), (5, 39), (6, 36)]
Nodo 1: Costo = 0
Nodo 2: Costo = 10
Nodo 3: Costo = 13
Nodo 4: Costo = 34
Nodo 5: Costo = 39
Nodo 6: Costo = 36
